# 流程

1. 首先手动登陆敦煌抢票网站https://www.mgk.org.cn/
2. 注册登录（每一个用户30天内仅能下一次订单）
3. 进入选票界面，正常票
4. **windows:** *ctrl+F12*    **Mac** *fn + F12*（打开开发者工具 上方tab选中到Network）
5. 选好时间，人次，点击检索场次，即可看到Network工具台出现多个请求，找到filed请求 点击右键选择**copy > copy as curl**
6. 将url复制到下方code block中
7. 然后将对应的信息填入到再下方的code block中，记住每次登陆失效后，只需要重新登录，将cookie替换即可
8. 具体选项看code block的注释
9. 总的来说网站没什么防御措施，比较简单，但是还是半自动的，有空升级成全自动

In [ ]:
curl 'https://www.mgk.org.cn/field' \
  -H 'Connection: keep-alive' \
  -H 'sec-ch-ua: " Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"' \
  -H 'Accept: */*' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Origin: https://www.mgk.org.cn' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Referer: https://www.mgk.org.cn/field' \
  -H 'Accept-Language: zh-CN,zh;q=0.9' \
  -H 'Cookie: aliyungf_tc=a30c56b7090b967980c05db8e4a76154eed84ee1f90fcb9faa85ac477f9f586d; HWWAFSESID=0f7c77359538d33c94; HWWAFSESTIME=1626442755442; Hm_lvt_43b18e3ab4b4ac9f3d1679e922eee451=1626442757; _uab_collina=162644277375420213778284; BENSESSCC_TAG=0f7c77359538d33c94; sendinfoDhPc=s%3AARpV6lXgaYkXFhBokvsRBL7v93Nnj5Ng.0BLoDzzpfmouu19hKwjAYobqZKoowv5ZlAll53F9R8U; userType=tourist; Hm_lpvt_43b18e3ab4b4ac9f3d1679e922eee451=1626493572' \
  --data-raw 'date=2021-08-09&langType=1&amount=5&ticketKind=0&baseCode=ticket_explain_A' \
  --compressed

In [ ]:
import urllib3
import json
import time
import random
import smtplib
from email.mime.text import MIMEText
from email.utils import formataddr

my_sender='xxx@qq.com'    # 发件人邮箱账号
my_pass = 'xxxxxx'        # 发件人邮箱密码 这个是在邮箱的账户中开启了SMTP服务获取的密钥，具体可以自行搜索方法（有空再更新）
my_user='xxx@qq.com'      # 收件人邮箱账号，我这边发送给自己

def requestByHttp():
  http = urllib3.PoolManager()
  requestBody = 'date=2021-08-06&langType=1&amount=1&ticketKind=0&baseCode=ticket_explain_A' # 这里是请求的场次的具体信息 「时间&语言（1为汉语）&参观人次（这里如果票紧张的话建议一张一张来，很少几率多张票放出来）&票类型&默认值」
  requestHeader = {
    'Connection':'keep-alive',
    'sec-ch-ua':'" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'Accept':'*/*',
    'X-Requested-With':'XMLHttpRequest',
    'sec-ch-ua-mobile':'?0',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin':'https://www.mgk.org.cn',
    'Sec-Fetch-Site':'same-origin',
    'Sec-Fetch-Mode':'cors',
    'Sec-Fetch-Dest':'empty',
    'Referer':'https://www.mgk.org.cn/field',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Cookie': 'aliyungf_tc=a30c56b7090b967980c05db8e4a76154eed84ee1f90fcb9faa85ac477f9f586d; HWWAFSESID=0f7c77359538d33c94; HWWAFSESTIME=1626442755442; Hm_lvt_43b18e3ab4b4ac9f3d1679e922eee451=1626442757; _uab_collina=162644277375420213778284; BENSESSCC_TAG=0f7c77359538d33c94; sendinfoDhPc=s%3AxG22vli7btHoUYetX31SAgVBgdalpkeT.61yferbBCJsacQvyPMhZmX2GYYrzv%2F59vBaue%2FJXma0; userType=tourist; Hm_lpvt_43b18e3ab4b4ac9f3d1679e922eee451=1626933870'
    # 每日要更换一下cookie，重新登录一下，因为要手机验证码，所以还是需要手动更换一下
  }
  #发起一个GET请求并且获取请求的响应结果
  r = http.request('POST', 'https://www.mgk.org.cn/field', body = requestBody, headers =requestHeader)
  #输出响应的数据
  try:
    status = json.loads(r.data)[0]['status']
    returnHtml = json.loads(r.data)[0]['html']
  except:
    print('被怀疑是刷票软件')
    randomeTime = random.randint(5,10) # 随机sleep，避免被当作刷票软件，具体阈值自己尝试，谨慎，容易给拉黑名单（换个网络即可）
    print('休息{}秒'.format(randomeTime))
    time.sleep(randomeTime) #休息两秒
    return 3
  else:
    # 如果没票的话就会出现这一串html， 目前将所有的非此串均当作失败
    nullText = '<!-- 场次信息--><h4 style="text-align:center;" class="section-title">场次信息</h4><table class="ticket-info-box"><tr><th align="left"><span class="align-l">日期</span></th><th>场次信息</th><th>语言</th><th align="right"><span class="align-r">操作</span></th></tr><tr><td colspan="4" class="no-data">暂无数据</td></tr></table>'
    print(status)
    if (status == 200 and returnHtml == nullText):
      return 2
    else:
      sendEmail(returnHtml)
      return 1

def sendEmail(htmlText): # 发邮件函数
  ret=True
  try:
      msg=MIMEText(htmlText,'html','utf-8')
      msg['From']=formataddr(["抢票脚本",my_sender]) 
      msg['To']=formataddr(["SAN",my_user]) 
      msg['Subject']="脚本抢到票啦！" 

      server=smtplib.SMTP_SSL("smtp.qq.com", 465) 
      server.login(my_sender, my_pass) 
      server.sendmail(my_sender,[my_user,],msg.as_string())  
      server.quit()  
  except Exception:
      ret=False
  return ret


if __name__ == '__main__':
  index=1
  while(True):
    status = requestByHttp()
    if (status == 1):
      print('有票啦！！！!    {}'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) ))
      time.sleep(120) # 抢到票了休息十分钟先
    elif(status == 2):
      print('还是没票    {}'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) ))
    else:
      print('怀疑是刷票软件，休息一下')
    randomeTime = random.randint(5,10)
    print('尝试第{0}次，休息{1}秒'.format(index, randomeTime))
    time.sleep(randomeTime)
    index += 1
